# Aventine: Demonstration

In [1]:
import os
import re
import json
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Union

import cltk
from cltk import NLP
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer

from aventine.library.config import ROOT_FINGERPRINT, CORPUS_FINGERPRINT
from aventine.library.config import ALLOWED_SYMBOLS, ALLOWED_PUNCTS
from aventine.library.config import ALLOWED_LEMMATA, BAD_LEMMATA
from aventine.library.config_ml import SENTENCE_TRANSFORMER_MODEL as ENG_MODEL
from aventine.library.config_ml import WORD_EMBEDDING_MODEL as LAT_MODEL
from aventine.library.utils import Checkpointer
from aventine.library.utils import meanings
from aventine.library.utils import normalise_text
from aventine.library.utils import get_null, replace_if_none
from aventine.library.files import perseus_url

c:\Users\vinny\.local\share\mamba\envs\aventine\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Indexed Data

In [2]:
sources_dir = Path('aventine/data/sources/')
index_dir = Path('aventine/data/dumps')
tool_dir = Path('aventine/tools/bin')

metadata_dir = sources_dir / 'metadata'

In [3]:
def get_metadata(text_id):
    with open(metadata_dir/f'{text_id}.json', 'r', encoding='utf-8') as f:
        return json.load(f)

In [4]:
root_ckpt = Checkpointer(index_dir/'root', ROOT_FINGERPRINT)
r = root_ckpt.load()

all_docs = [i for i in os.listdir(index_dir) if i != 'root']
title2id = {get_metadata(text_id)['title']: text_id for text_id in all_docs} | {'root': 'root'}
titles2ids = lambda x: [title2id[i] for i in x]

In [5]:
text_ckpts = {
    text_id: Checkpointer(index_dir/text_id, CORPUS_FINGERPRINT).load()
    for text_id in all_docs
}
text_metas = {
    text_id: get_metadata(text_id)
    for text_id in all_docs
}

In [6]:
root_lat_embeddings = np.array(r.lat_embeddings, copy=False)
root_eng_embeddings = np.array(r.eng_embeddings, copy=False)
root_lemmata_arr = np.array(r.lemmata_arr, copy=False)
root_definitions = np.array(r.definitions, copy=False)

In [7]:
lat_model =  LAT_MODEL('lat')
eng_model = SentenceTransformer(ENG_MODEL, trust_remote_code=True)
lat_none, _ = get_null(lat_model, eng_model)

if LAT_MODEL is cltk.embeddings.embeddings.FastTextEmbeddings:
    lat_fallback = cltk.embeddings.embeddings.Word2VecEmbeddings('lat')
elif LAT_MODEL is cltk.embeddings.embeddings.Word2VecEmbeddings:
    lat_fallback = cltk.embeddings.embeddings.FastTextEmbeddings('lat')

cltk_nlp = NLP(language="lat")
cltk_nlp.pipeline.processes = [
    cltk.alphabet.processes.LatinNormalizeProcess,
    cltk.dependency.processes.LatinStanzaProcess
]

‎𐤀 CLTK version '1.4.0'. When using the CLTK in research, please cite: https://aclanthology.org/2021.acl-demo.3/

Pipeline for language 'Latin' (ISO: 'lat'): `LatinNormalizeProcess`, `LatinStanzaProcess`, `LatinEmbeddingsProcess`, `StopsProcess`, `LatinLexiconProcess`.

⸖ ``LatinStanzaProcess`` using Stanza model from the Stanford NLP Group: https://stanfordnlp.github.io/stanza/ . Please cite: https://arxiv.org/abs/2003.07082
⸖ ``LatinEmbeddingsProcess`` using word2vec model by University of Oslo from http://vectors.nlpl.eu/ . Please cite: https://aclanthology.org/W17-0237/
⸖ ``LatinLexiconProcess`` using Lewis's *An Elementary Latin Dictionary* (1890).

⸎ To suppress these messages, instantiate ``NLP()`` with ``suppress_banner=True``.


## Inference Functions

In [8]:
def get_similarities(a, vects):
    sims = vects @ a.transpose()
    norm_a = np.linalg.norm(a)
    norm_v = np.linalg.norm(vects, axis=1)
    revalued = sims / (2 * norm_a * norm_v) + 0.5
    return revalued

def argmaxk(arr, k):
    return np.flip(np.argsort(arr)[-k:])

def atomise(query):
    query = re.sub(ALLOWED_PUNCTS, '', query)
    query = normalise_text(query, ALLOWED_SYMBOLS, ALLOWED_PUNCTS)
    cltk_nlp(text=query)
    atoms = [w.strip() for w in query.split(' ')
             if re.fullmatch(ALLOWED_LEMMATA, w) and w not in BAD_LEMMATA]
    return atoms

In [31]:
class Word2VecWrapper():
    def __init__(self, model):
        self.model = model
    
    def get_word_vector(self, word):
        try:
            return self.model.wv[word]
        except KeyError:
            return None
    

def search(query,
           language,
           results: int = 50,
           texts: list = all_docs,
           scope: Union["universal", "root", str] = "universal",
           tool_dir: Path = tool_dir):
    
    texts = set(texts)
    
    if language == 'eng':
        sent = eng_model.encode(query)
        sims = get_similarities(sent, root_eng_embeddings)
        lemmata = root_lemmata_arr
        repeated = set()
    
    elif language == 'lat':
        
        if scope == 'universal':
            embedder = lat_model
            lemmata = root_lemmata_arr
            vects = root_lat_embeddings
        else:
            wv_model = Word2Vec.load(str(index_dir/scope/'word2vec.model'))
            embedder = Word2VecWrapper(wv_model)
            lemmata, vects = [], []
            for k in wv_model.wv.key_to_index:
                lemmata.append(k)
                vects.append(wv_model.wv[k])
            lemmata, vects = np.array(lemmata, copy=False), np.array(vects, copy=False)
        
        senses = []
        atoms = atomise(query)
        repeated = set(atoms)
        for w in atoms:
            arr = embedder.get_word_vector(w)
            if arr is not None:
                senses.append(arr)
        if senses == []:
            warnings.warn("No valid lemmata found in the query.")
            return None
        
        sent = np.mean(senses, axis=0)
        sims = get_similarities(sent, vects)
    

    # Given arbitrary arrays `lemma` and `sims`
    sorted = np.flip(np.argsort(sims))
    found = 0
    idx = 0
    data = []
    
    while found < results and idx < len(sorted):
        lemma_idx = sorted[idx]
        lemma = lemmata[lemma_idx]

        if lemma in repeated:
            idx += 1
            continue

        if len(texts) == 0:
            meaning = root_definitions[lemma_idx] if lemma in r.existing_lemmata \
                      else meanings(lemma, tool_dir=tool_dir)
            data.append([sims[lemma_idx], lemma, meaning, '', ''])
            idx += 1
            found += 1

        elif lemma in r.existing_lemmata:
            intersect = r.root_lemmata_info[lemma]['texts'].intersection(texts)
            if intersect:
                found += 1
                urls = []
                for text_id in intersect:
                    quotes = text_ckpts[text_id].corpus_lemmata_info[lemma]['loc']
                    urls.extend([perseus_url(text_metas[text_id],
                                            text_metas[text_id]['index'][quote_id])
                                for quote_id in quotes])
                data.append([sims[lemma_idx],
                            lemma,
                            root_definitions[lemma_idx],
                            ', '.join([text_metas[text_id]['title'] for text_id in intersect]),
                            ', '.join(urls)])
        
        idx += 1

    df = pd.DataFrame(data, columns=['Score', 'Latin Word', 'Definition', 'Texts', 'Links'])
    return df

In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Inference Demo

In [32]:
df = search('ceramics and pottery; vases',
            language='eng',
            texts=titles2ids(['Aeneid', 'Naturalis Historia', 'Annales']),
            results=10)
df

Score Latin Word  \
0  0.850927      testa   
1  0.850927     testas   
2  0.848908    amphura   
3  0.848908    amphora   
4  0.848908     amphus   
5  0.844228   fictilus   
6  0.840010     phiala   
7  0.839132    argilla   
8  0.839132   argillus   
9  0.838985    figulus   

                                                                      Definition  \
0  object made from burnt clay; earthenware jar; fragment of earthenware, shard;   
1  object made from burnt clay; earthenware jar; fragment of earthenware, shard;   
2       amphora, pitcher, two handled earthenware jar; a capacity of ~30 liters;   
3       amphora, pitcher, two handled earthenware jar; a capacity of ~30 liters;   
4       amphora, pitcher, two handled earthenware jar; a capacity of ~30 liters;   
5                                         of clay; made of earthenware, earthen;   
6                                               drinking vessel; a bowl, saucer;   
7                                         white clay, potter's earth/clay; clay;   
8                                         white clay, potter's earth/clay; clay;   
9                                          potter; maker of earthenware vessels;   

                Texts  \
0  Naturalis Historia   
1  Naturalis Historia   
2  Naturalis Historia   
3  Naturalis Historia   
4  Naturalis Historia   
5  Naturalis Historia   
6  Naturalis Historia   
7  Naturalis Historia   
8  Naturalis Historia   
9  Naturalis Historia   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
search('phiala',
       language='lat',
       texts=titles2ids(['Aeneid', 'Naturalis Historia', 'Annales']),
       results=10)

,Score,Latin Word,Definition,Texts,Links
0,0.879760,selibra,half-pound;,Naturalis Historia,"https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+14.38.107&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+18.26.66&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+18.28.72&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+18.34.89&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+23.52.108&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+34.36.94&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+35.13.36&fromdoc=Perseus:text:1999.02.0138"
1,0.873529,acetabulum,small cup (esp. for vinegar); (1/8 pint); cup-shaped part (plant); hip joint;,Naturalis Historia,"https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+18.28.72&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+26.32.62&fromdoc=Perseus:text:1999.02.0138"
2,0.865911,medimnus,"dry measure, Greek bushel (6 modii); measure of land in Cyrenaica;",Naturalis Historia,https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+6.58.189&fromdoc=Perseus:text:1999.02.0138
3,0.850259,cados,"jar, large jar for wine/oil/liquids; urn, funeral urn; money jar (L+S);",Naturalis Historia,"https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+12.54.132&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+13.9.39&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+14.26.80&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+19.45.139&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+22.57.119&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+36.49.157&fromdoc=Perseus:text:1999.02.0138"
4,0.848309,quinquagena,50 each/apiece/times/fold/together/at a time - 'how many each'; by 50s;,Naturalis Historia,"https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+33.20.58&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+33.23.67&fromdoc=Perseus:text:1999.02.0138"
5,0.843770,nardinus,wine flavored with nard (an aromatic plant); of/pertaining to nard (an aromatic plant); resembling/smelling like nard; nard-,Naturalis Historia,https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+15.17.53&fromdoc=Perseus:text:1999.02.0138
6,0.842724,vngula,hoof; bird claw/talon; (torture); toe nail; pig's foot/trotter (food/medicine);,Naturalis Historia,https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+11.111.255&fromdoc=Perseus:text:1999.02.0138
7,0.841487,paniculus,plant tuft; piece of thatch; B:swelling;,Naturalis Historia,"https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+8.112&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+16.28.45&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+18.19.53&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+18.19.53&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+21.11.16&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+32.57.140&fromdoc=Perseus:text:1999.02.0138, https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+35.64.178&fromdoc=Perseus:text:1999.02.0138"
8,0.838028,dcccclx,980 as a ROMAN NUMERAL;,Naturalis Historia,https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+4.35.89&fromdoc=Perseus:text:1999.02.0138
9,0.834968,lineus,"smear, plaster (with); seal (wine jar); erase/rub over; befoul; cover/overlay;",Naturalis Historia,https://www.perseus.tufts.edu/hopper/text?doc=Plin.+Nat.+29.40.112&fromdoc=Perseus

In [35]:
search('phiala',
       language='lat',
       texts=[],
       results=10)

,Score,Latin Word,Definition,Texts,Links
0,0.879760,selibra,half-pound;,,
1,0.865911,medimnus,"dry measure, Greek bushel (6 modii); measure of land in Cyrenaica;",,
2,0.848309,quinquagena,50 each/apiece/times/fold/together/at a time - 'how many each'; by 50s;,,
3,0.842724,vngula,hoof; bird claw/talon; (torture); toe nail; pig's foot/trotter (food/medicine);,,
4,0.838028,dcccclx,980 as a ROMAN NUMERAL;,,
5,0.834968,lineus,"smear, plaster (with); seal (wine jar); erase/rub over; befoul; cover/overlay;",,
6,0.832550,recrementum,"increase, growth; [root/prefix/suffix information: - back, - again;]",,
7,0.831404,faginus,"of the beech tree; of beech-wood, beechen;",,
8,0.830529,amphora,"amphora, pitcher, two handled earthenware jar; a capacity of ~30 liters;",,
9,0.829623,trecenties,"300 times, on 300 occasions - (ADVERB answers 'how often');",,
